In [1]:
import json
import pandas as pd
import numpy as np
import random
import modules.gnrl as gnrl
import modules.LP_Pool_State as lpps
import datetime




In [2]:
with open("config.json", 'r') as f:
    args = json.load(f)
with open("MP_ASSET_STATE.csv","r") as outfile:
    MP_Asset_State = pd.read_csv(outfile, index_col=0)
with open("MP_ASSET.csv","r") as outfile:
    MP_Asset = pd.read_csv(outfile, index_col=0)

LS_Opening = pd.read_csv("LS_Opening",index_col=0)

In [3]:
#def timestamps(LS_Opening,args):
#    return
#    dates = LS_Opening['LS_timestamp'].astype("datetime64").tolist()
##    time = LS_Opening["SYS_LS_expected_payment"] + LS_Opening["SYS_LS_expected_penalty"]
#    timestamp = pd.DataFrame({"Dates":[]})
#    print(len(time))
#    i=0
#    for i in range(len(time)):
#        s = pd.date_range(start=dates[i],end=dates[i]+pd.Timedelta(days=time[i]*30),freq="M")
#        s = pd.DataFrame(s)
#        s = s.explode(0)
#        print(s)
#        timestamp = pd.concat([timestamp,s],ignore_index=True)
#        i+=1
#    print(timestamp)
#    return timestamp
#timestamp = timestamps(LS_Opening,args)

In [4]:
def lsr_timestamp(LS_Opening,args):
    month = 2592000
    duration = LS_Opening["SYS_LS_expected_payment"] + LS_Opening["SYS_LS_expected_penalty"]
    timestamp = pd.DataFrame({"LS_timestamp":LS_Opening["LS_timestamp"],"LS_contract_id":LS_Opening["LS_contract_id"]})
    timestamp["LS_timestamp"] = LS_Opening["LS_timestamp"].astype("datetime64").apply(lambda x:datetime.datetime.timestamp(x)).astype("uint64")
    timestamp["LP_Pool_id"] = LS_Opening["LP_Pool_id"]
    
    timestamp = pd.DataFrame(np.repeat(timestamp.values, duration, axis=0))
    timestamp = pd.DataFrame({"LS_timestamp":timestamp[0],"LS_contract_id":timestamp[1],"LP_Pool_id":timestamp[2]})
    timestamp["LS_timestamp"] = timestamp["LS_timestamp"].values.astype("uint64")
    #duration = pd.DataFrame({"duration":duration})
    d = []
    l1 = []
    for i in duration:
        l1 = [s for s in range(1,i+1)]
        d.extend(l1)
    timestamp["Duration"] = d 
    timestamp["Duration"] = timestamp["Duration"]*month
    timestamp["LS_timestamp"] = timestamp["LS_timestamp"]+timestamp["Duration"]
    timestamp["LS_timestamp"] = timestamp["LS_timestamp"].apply(datetime.datetime.fromtimestamp)
    timestamp = timestamp.drop("Duration",axis=1)
    
    return timestamp



In [5]:
def LS_Repayment_generate(LS_Opening,pool_id,args):#WRONG  witthout penalty add column flag if 1  =  payment if 2 penalty 
     

    LS_Repayment = pd.DataFrame(lsr_timestamp(LS_Opening,args))
    lp_dep_h = [s for s in range(100,100+len(LS_Repayment))]
    lp_dep_idx = lp_dep_h
    LS_Repayment["LS_repayment_height"] = lp_dep_h
    LS_Repayment["LS_repayment_idx"] = lp_dep_idx
    LS_Repayment = pd.merge(LS_Repayment,pool_id,on="LP_Pool_id",how='left')
    LS_Repayment = LS_Repayment.rename(columns={"LP_symbol":"LS_symbol"})
    LS_Repayment = LS_Repayment.drop("LP_Pool_id",axis=1)
    LS_Repayment["LS_amnt_stable"] = 0
    LS_Repayment["LS_principal_stable"] = 0
    LS_Repayment["LS_current_margin_stable"] = 0
    LS_Repayment["LS_current_interest_stable"] = 0
    LS_Repayment["LS_prev_interest_stable"] = 0
    LS_Repayment["LS_prev_margin_stable"] = 0

    
    return LS_Repayment


In [6]:
pool_id = lpps.LP_pool_gen(args)
LS_Repayment = LS_Repayment_generate(LS_Opening,pool_id,args)
#LS_Repayment.sample(20)


In [8]:
close

,LS_timestamp,LS_contract_id
0,2020-11-20 02:00:00,LScid1000000
1,2020-10-21 03:00:00,LScid1000001
2,2020-12-20 02:00:00,LScid1000002
3,2020-10-21 03:00:00,LScid1000003
4,2021-09-16 03:00:00,LScid1000004
...,...,...
73043,2022-09-20 03:00:00,LScid1073043
73044,2022-09-20 03:00:00,LScid1073044
73045,2023-01-18 02:00:00,LScid1073045
73046,2023-02-17 02:00:00,LScid1073046


In [9]:
def LS_Liquidation_generate(LS_Opening,LS_Repayment,args):#WRONG form it ONLY from customers with penalty! 
    contracts = LS_Opening.loc[LS_Opening["SYS_LS_expected_penalty"]>=1]["LS_contract_id"]
    LS_Liquidation = LS_Repayment.loc[LS_Repayment["LS_contract_id"].isin(contracts)]
    LS_Liquidation["LS_transaction_type"] = 1
    LS_Liquidation=LS_Liquidation.drop(["LS_repayment_height","LS_repayment_idx"],axis=1)
    lp_dep_h = [s for s in range(100,100+len(LS_Liquidation))]
    lp_dep_idx = lp_dep_h
    LS_Liquidation["LS_liquidation_height"] = lp_dep_h
    LS_Liquidation["LS_liquidation_idx"] = lp_dep_idx
    LS_Liquidation["LS_amnt_stable"] = 0
    LS_Liquidation["LS_principal_stable"] = 0
    LS_Liquidation["LS_current_margin_stable"] = 0
    LS_Liquidation["LS_current_interest_stable"] = 0
    LS_Liquidation["LS_prev_interest_stable"] = 0
    LS_Liquidation["LS_prev_margin_stable"] = 0
    return LS_Liquidation
liq = LS_Liquidation_generate(LS_Opening,LS_Repayment,args)

/tmp/ipykernel_20779/2261401898.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LS_Liquidation["LS_transaction_type"] = 1


In [10]:
liq

,LS_timestamp,LS_contract_id,LS_symbol,LS_amnt_stable,LS_principal_stable,LS_current_margin_stable,LS_current_interest_stable,LS_prev_interest_stable,LS_prev_margin_stable,LS_transaction_type,LS_liquidation_height,LS_liquidation_idx
5,2020-09-21 03:00:00,LScid1000002,usd-coin,0,0,0,0,0,0,1,100,100
6,2020-10-21 03:00:00,LScid1000002,usd-coin,0,0,0,0,0,0,1,101,101
7,2020-11-20 02:00:00,LScid1000002,usd-coin,0,0,0,0,0,0,1,102,102
8,2020-12-20 02:00:00,LScid1000002,usd-coin,0,0,0,0,0,0,1,103,103
11,2020-09-21 03:00:00,LScid1000004,tether,0,0,0,0,0,0,1,104,104
...,...,...,...,...,...,...,...,...,...,...,...,...
391236,2023-12-21 02:00:00,LScid1067650,tether,0,0,0,0,0,0,1,234811,234811
397083,2022-08-11 03:00:00,LScid1069038,usd-coin,0,0,0,0,0,0,1,234812,234812
397084,2022-09-10 03:00:00,LScid1069038,usd-coin,0,0,0,0,0,0,1,234813,234813
397085,2022-10-10 03:00:00,LScid1069038,usd-coin,0,0,0,0,0,0,1,234814,234814


In [11]:
liq.to_csv("LS_Liquidation")
close.to_csv("LS_Closing")
LS_Repayment.to_csv("LS_Repayment")

In [12]:
LS_Opening


,LS_asset_symbol,LS_timestamp,LS_contract_id,LS_address_id,LS_loan_symbol,LS_loan_amnt_asset,LS_loan_amnt_stable,LS_cltr_symbol,LS_cltr_amnt_stable,LS_cltr_amnt_asset,SYS_LS_cltr_amnt_asset,SYS_LS_loan_amnt_asset,LS_native_amnt_nolus,LS_native_amnt_stable,SYS_LS_expected_payment,SYS_LS_expected_penalty,LP_Pool_id,LS_interest
0,solana,2020-08-22 03:00:00,LScid1000000,LS_aid1000000,"('solana', None, 16)",146120851640169,4283606072121,solana,5936057173648,202488677762287,2.024887,1.461209,0,0,3,0,pid101,NaN
1,ethereum,2020-08-22 03:00:00,LScid1000001,LS_aid1000001,"('ethereum', None, 13)",8041181306339,310878269545215,ethereum,394108143559885,10194006294830,1.019401,0.804118,0,0,2,0,pid101,NaN
2,usd-coin,2020-08-22 03:00:00,LScid1000002,LS_aid1000002,"('usd-coin', None, 16)",501203184993,501020135184,usd-coin,659006249718,659247020416,0.659247,0.501203,0,0,3,1,pid100,NaN
3,matic-network,2020-08-22 03:00:00,LScid1000003,LS_aid1000003,"('matic-network', None, 16)",10738170597123684,26797738702,matic-network,39790148758,15944382852572164,1.594438,1.073817,0,0,2,0,pid101,NaN
4,crypto-com-chain,2020-08-22 03:00:00,LScid1000004,LS_aid1000004,"('crypto-com-chain', None, 15)",6969856732475531,117280371682,crypto-com-chain,121202182911,7202926102989536,0.720293,0.696986,0,0,5,8,pid101,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73043,tether,2022-08-21 03:00:00,LScid1073043,LS_aid1073043,"('tether', None, 12)",1158363644598,1159646859896,tether,1190204761684,1188887732327,1.188888,1.158364,0,0,1,0,pid101,NaN
73044,bitcoin,2022-08-21 03:00:00,LScid1073044,LS_aid1073044,"('bitcoin', None, 12)",1233262442095,26122219920255380,bitcoin,26956457674587960,1272647842469,1.272648,1.233262,0,0,1,0,pid101,NaN
73045,wrapped-bitcoin,2022-08-21 03:00:00,LScid1073045,LS_aid1073045,"('wrapped-bitcoin', None, 12)",1522935880862,32224525557920664,wrapped-bitcoin,35167825880797500,1662036692814,1.662037,1.522936,0,0,5,0,pid101,NaN
73046,tether,2022-08-21 03:00:00,LScid1073046,LS_aid1073046,"('tether', None, 12)",1236486906930,1237171596812,tether,1280949110456,1280240192713,1.280240,1.236487,0,0,6,0,pid100,NaN


In [16]:
contracts = LS_Opening.loc[LS_Opening["SYS_LS_expected_penalty"]<1]["LS_contract_id"]
LS_Liquidation = LS_Repayment.loc[LS_Repayment["LS_contract_id"].isin(contracts)]

In [17]:
LS_Repayment.loc[LS_Repayment["LS_contract_id"].isin(contracts)].value_counts()

LS_timestamp         LS_contract_id  LS_repayment_height  LS_repayment_idx  LS_symbol
2020-08-05 03:00:00  LScid1000002    113                  113               tether       1
2022-04-26 03:00:00  LScid1067504    461980               461980            usd-coin     1
                     LScid1067534    462135               462135            usd-coin     1
                     LScid1067535    462142               462142            tether       1
                     LScid1067536    462147               462147            tether       1
                                                                                        ..
2021-07-06 03:00:00  LScid1029071    202167               202167            usd-coin     1
                     LScid1029072    202173               202173            tether       1
                     LScid1029073    202179               202179            usd-coin     1
                     LScid1029074    202184               202184            usd-coin     1
2023

In [14]:
LS_Repayment

,LS_timestamp,LS_contract_id,LS_repayment_height,LS_repayment_idx,LS_symbol
0,2020-08-05 03:00:00,LScid1000000,100,100,usd-coin
1,2020-09-04 03:00:00,LScid1000000,101,101,usd-coin
2,2020-10-04 03:00:00,LScid1000000,102,102,usd-coin
3,2020-11-03 02:00:00,LScid1000000,103,103,usd-coin
4,2020-12-03 02:00:00,LScid1000000,104,104,usd-coin
...,...,...,...,...,...
511576,2022-09-03 03:00:00,LScid1077542,511676,511676,usd-coin
511577,2022-10-03 03:00:00,LScid1077542,511677,511677,usd-coin
511578,2022-11-02 02:00:00,LScid1077542,511678,511678,usd-coin
511579,2022-12-02 02:00:00,LScid1077542,511679,511679,usd-coin


In [15]:
511581 - 280191

231390

In [18]:
LS_Liquidation

,LS_timestamp,LS_contract_id,LS_repayment_height,LS_repayment_idx,LS_symbol
13,2020-08-05 03:00:00,LScid1000002,113,113,tether
14,2020-09-04 03:00:00,LScid1000002,114,114,tether
15,2020-10-04 03:00:00,LScid1000002,115,115,tether
65,2020-08-05 03:00:00,LScid1000010,165,165,usd-coin
66,2020-09-04 03:00:00,LScid1000010,166,166,usd-coin
...,...,...,...,...,...
511576,2022-09-03 03:00:00,LScid1077542,511676,511676,usd-coin
511577,2022-10-03 03:00:00,LScid1077542,511677,511677,usd-coin
511578,2022-11-02 02:00:00,LScid1077542,511678,511678,usd-coin
511579,2022-12-02 02:00:00,LScid1077542,511679,511679,usd-coin
